<a href="https://colab.research.google.com/github/wkd-woo/finance-statements/blob/main/%ED%8F%AC%EA%B4%84%EC%86%90%EC%9D%B5%EC%9D%BC%EA%B4%84.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U finance-datareader

In [2]:
import pandas as pd
import os
from tqdm import tqdm
import FinanceDataReader as fdr

# 파일별로 분류

In [3]:
import glob

재무상태표 = [] # _01_
손익계산서 = [] # _02_
포괄손익계산서 = [] # _03_
현금흐름표 = [] # _04_
자본변동표 = [] #_05_


for name in glob.glob('/content/drive/MyDrive/22캡스톤디자인/재무일괄/*.txt'):
  if '_01_' in name:
    재무상태표.append(name)
  elif '_02_' in name:
    손익계산서.append(name)
  elif '_03_' in name:
    포괄손익계산서.append(name)
  elif '_04_' in name:
    현금흐름표.append(name)
  elif '_05_' in name:
    자본변동표.append(name)

In [71]:
sample = pd.DataFrame(columns=['종목코드','사업연도', '보고서종류', '회사명', '업종', '항목코드','항목명', '당기'])

# 데이터 추출

In [72]:
for path in tqdm(포괄손익계산서):
  df = pd.read_csv(path, delimiter = '\t', encoding='cp949')

  당기 = ''
  if df['보고서종류'][0] == '1분기보고서':
    당기 = '당기 1분기 누적' 
  elif df['보고서종류'][0] == '반기보고서':
    당기 = '당기 반기 누적' 
  elif df['보고서종류'][0] == '3분기보고서':
    당기 = '당기 3분기 누적' 
  elif df['보고서종류'][0] == '사업보고서':
    당기 = '당기' 

  df['사업연도'] = path[52:56] # 사업연도 컬럼 설정 - 파일명에서 가져온다
  df['항목명'] = df['항목명'].str.strip() # 항목명 컬럼 문자열 전처리
  df['항목코드'] = df['항목코드'].str.strip() # 항목코드 컬럼 문자열 전처리

  df = df[['종목코드','사업연도', '보고서종류', '회사명', '업종', '항목코드','항목명', 당기]]

  전체종목 = len(df['종목코드'].unique())

  # (수익)매출액 (ifrs-full_Revenue)
  매출액 = df[(df['항목코드'] == 'ifrs_Revenue') | (df['항목코드'] == 'ifrs-full_Revenue') | (df['항목명'].isin(['매출액','(수익)매출액','매출액(수익)','수익(매출액)']))]
  매출액.rename(columns={당기:"당기"}, inplace=True)
  매출액.drop_duplicates(['종목코드','사업연도','보고서종류','항목명'])
  sample = sample.append(매출액, sort=False, ignore_index=True)
  

  # 매출총이익 (ifrs-full_GrossProfit))
  매출총이익 = df[(df['항목코드'] == 'ifrs_GrossProfit') | (df['항목코드'] == 'ifrs-full_GrossProfit')| (df['항목명'].isin(['매출총이익','(수익)매출총이익','매출총이익(손실)']))]
  매출총이익.rename(columns={당기:"당기"}, inplace=True)
  매출총이익.drop_duplicates(['종목코드','사업연도','보고서종류','항목명'])
  sample = sample.append(매출총이익, sort=False, ignore_index=True)

  # 영업이익(손실) (dart_OperatingIncomeLoss)
  영업이익 = df[(df['항목코드'] == 'dart_OperatingIncomeLoss')|(df['항목명'].isin(['영업이익','영업이익(손실)']))]
  영업이익.rename(columns={당기:"당기"}, inplace=True)
  영업이익.drop_duplicates(['종목코드','사업연도','보고서종류','항목명'])
  sample = sample.append(영업이익, sort=False, ignore_index=True)
  
  # 당기순이익(손실) (ifrs_ProfitLoss)
  당기순이익 = df[(df['항목코드'] == 'ifrs-full_ProfitLoss') | (df['항목코드'] == 'ifrs_ProfitLoss') |  (df['항목명'].isin(['당기순이익','당기순이익(손실)']))]
  당기순이익.rename(columns={당기:"당기"}, inplace=True)
  당기순이익.drop_duplicates(['종목코드','사업연도','보고서종류','항목명'])
  sample = sample.append(당기순이익, sort=False, ignore_index=True)

  # 감가상각비(dart_DepreciationExpense)
  감가상각비 = df[(df['항목코드'] == 'dart_DepreciationExpense')|(df['항목명'].isin(['감가상각비']))]
  감가상각비.rename(columns={당기:"당기"}, inplace=True)
  sample = sample.append(감가상각비, sort=False, ignore_index=True)

  # 법인세비용차감전순이익(손실)(ifrs-full_ProfitLossBeforeTax)
  법인세차감전순이익 = df[(df['항목코드'] == 'ifrs-full_ProfitLossBeforeTax') | (df['항목코드'] == 'ifrs_ProfitLossBeforeTax') | df['항목명'].isin(['법인세차감전순이익','법인세차감전순이익(손실)'])]
  법인세차감전순이익.rename(columns={당기:"당기"}, inplace=True)
  법인세차감전순이익.drop_duplicates(['종목코드','사업연도','보고서종류','항목명'])
  sample = sample.append(법인세차감전순이익, sort=False, ignore_index=True)
  


  0%|          | 0/24 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
100%|██████████| 24/24 [00:12<00:00,  1.90it/s]


# 데이터 전처리

In [73]:
sample['종목코드'] = sample['종목코드'].apply(lambda x:x[1:-1])

In [81]:
sample['항목명'] = sample.apply(lambda x: '매출액' if x['항목코드'] == 'ifrs_Revenue' else x['항목명'], axis=1)
sample['항목명'] = sample.apply(lambda x: '매출액' if x['항목코드'] == 'ifrs-full_Revenue' else x['항목명'], axis=1)
sample['항목명'] = sample.apply(lambda x: '매출총이익' if x['항목코드'] == 'ifrs_GrossProfit' else x['항목명'], axis=1)
sample['항목명'] = sample.apply(lambda x: '매출총이익' if x['항목코드'] == 'ifrs-full_GrossProfit' else x['항목명'], axis=1)
sample['항목명'] = sample.apply(lambda x: '영업이익' if x['항목코드'] == 'dart_OperatingIncomeLoss' else x['항목명'], axis=1)
sample['항목명'] = sample.apply(lambda x: '당기순이익' if x['항목코드'] == 'ifrs_ProfitLoss' else x['항목명'], axis=1)
sample['항목명'] = sample.apply(lambda x: '당기순이익' if x['항목코드'] == 'ifrs-full_ProfitLoss' else x['항목명'], axis=1)
sample['항목명'] = sample.apply(lambda x: '감가상각비' if x['항목코드'] == 'dart_DepreciationExpense' else x['항목명'], axis=1)
sample['항목명'] = sample.apply(lambda x: '법인세비용차감전순이익' if x['항목코드'] == 'ifrs_ProfitLossBeforeTax' else x['항목명'], axis=1)
sample['항목명'] = sample.apply(lambda x: '법인세비용차감전순이익' if x['항목코드'] == 'ifrs-full_ProfitLossBeforeTax' else x['항목명'], axis=1)
sample['항목명'] = sample.apply(lambda x: '매출액' if x['항목명'] == '수익(매출액)' else x['항목명'], axis=1)
sample['항목명'] = sample.apply(lambda x: '매출총이익' if x['항목명'] == '매출총이익(손실)' else x['항목명'], axis=1)
sample['항목명'] = sample.apply(lambda x: '영업이익' if x['항목명'] == '영업이익(손실)' else x['항목명'], axis=1)
sample['항목명'] = sample.apply(lambda x: '당기순이익' if x['항목명'] == '당기순이익(손실)' else x['항목명'], axis=1)
sample['항목명'] = sample.apply(lambda x: '법인세차감전순이익' if x['항목명'] == '법인세차감전순이익(손실)' else x['항목명'], axis=1)

sample

,종목코드,사업연도,보고서종류,회사명,업종,항목코드,항목명,당기
0,060310,2016,반기보고서,3S,262,ifrs_Revenue,매출액,"13,824,059,417"
1,095570,2016,반기보고서,AJ네트웍스,693,ifrs_Revenue,매출액,"599,785,915,747"
2,006840,2016,반기보고서,AK홀딩스,715,ifrs_Revenue,매출액,"1,378,197,891,715"
3,054620,2016,반기보고서,AP시스템,292,ifrs_Revenue,매출액,"148,087,692,662"
4,027410,2016,반기보고서,BGF리테일,471,ifrs_Revenue,매출액,"2,364,759,516,212"
...,...,...,...,...,...,...,...,...
174285,069260,2016,3분기보고서,휴켐스,204,ifrs_ProfitLossBeforeTax,법인세비용차감전순이익,"45,234,164,953"
174286,010240,2016,3분기보고서,흥국,292,ifrs_ProfitLossBeforeTax,법인세비용차감전순이익,"3,483,104,424"
174287,189980,2016,3분기보고서,흥국에프엔비,112,ifrs_ProfitLossBeforeTax,법인세비용차감전순이익,"4,830,562,718"
174288,003280,2016,3분기보고서,흥아해운,501,ifrs_ProfitLossBeforeTax,법인세비용차감전순이익,"-27,534,283,994"


In [82]:
sample['항목명'].unique()

array(['매출액', '매출총이익', '영업이익', '당기순이익', '감가상각비', '법인세비용차감전순이익',
       '법인세차감전순이익'], dtype=object)

In [83]:
포괄손익일괄 = sample

# 저장

In [84]:
포괄손익일괄.to_csv('/content/drive/MyDrive/22캡스톤디자인/재무일괄/일괄/포괄손익일괄.csv')